<a href="https://colab.research.google.com/github/agonzalezl2025/Parcial2/blob/main/Parcial2HE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de datos
